In [ ]:
import numpy as np
import networkx as nx
from networkx.algorithms.matching import max_weight_matching
from sklearn.cluster import DBSCAN

def merge_similar_nodes(sample, eps=0.1):
    clustering = DBSCAN(eps=eps, min_samples=1).fit(sample)
    labels = clustering.labels_
    unique_labels = np.unique(labels)
    node_map = {old_idx: new_idx for new_idx, label in enumerate(unique_labels) for old_idx in np.where(labels == label)[0]}

    print(f"Labels: {labels}")
    return node_map, labels

def compute_sequence_distance_matrix_with_jumps(sequence_length, labels, jump_cost=0.1):
    dist_matrix = np.abs(np.subtract.outer(np.arange(sequence_length), np.arange(sequence_length)))

    # Adjust distances for jumps between similar nodes
    for i in range(sequence_length):
        for j in range(i+1, sequence_length):
            if labels[i] == labels[j]:
                dist_matrix[i, j] = dist_matrix[j, i] = jump_cost

    print(f"Distance Matrix:\n{dist_matrix}")
    return dist_matrix

def compute_mst(dist_matrix):
    G = nx.Graph()
    sequence_length = dist_matrix.shape[0]
    for i in range(sequence_length):
        for j in range(i+1, sequence_length):
            G.add_edge(i, j, weight=dist_matrix[i, j])
    mst = nx.minimum_spanning_tree(G)

    print(f"MST Edges: {list(mst.edges(data=True))}")
    return mst

def find_odd_degree_vertices(mst):
    odd_degree_nodes = [v for v in mst.nodes() if mst.degree(v) % 2 != 0]

    print(f"Odd Degree Nodes: {odd_degree_nodes}")
    return odd_degree_nodes

def compute_min_weight_matching(subgraph, dist_matrix):
    min_weight_matching = max_weight_matching(subgraph, maxcardinality=True)
    matching = nx.Graph()
    for u, v in min_weight_matching:
        matching.add_edge(u, v, weight=dist_matrix[u, v])

    print(f"Matching Edges: {list(matching.edges(data=True))}")
    return matching

def find_eulerian_circuit(eulerian_multigraph):
    eulerian_circuit = list(nx.eulerian_circuit(eulerian_multigraph))

    print(f"Eulerian Circuit: {eulerian_circuit}")
    return eulerian_circuit

def convert_to_hamiltonian_circuit(eulerian_circuit):
    hamiltonian_path = []
    visited = set()
    for u, v in eulerian_circuit:
        if u not in visited:
            hamiltonian_path.append(u)
            visited.add(u)
    if hamiltonian_path[0] != hamiltonian_path[-1]:
        hamiltonian_path.append(hamiltonian_path[0])

    print(f"Hamiltonian Path: {hamiltonian_path}")
    return hamiltonian_path

def christofides_algorithm(samples, eps=0.1, jump_cost=0.1):
    sample_size, sequence_length, feature_size = samples.shape
    all_tours = []
    for sample_index, sample in enumerate(samples):
        print(f"\nProcessing Sample {sample_index + 1}/{sample_size}")

        # Step 1: Merge similar nodes
        node_map, labels = merge_similar_nodes(sample, eps)

        # Step 2: Create a new sample array with merged nodes
        merged_sample = np.array([sample[labels == label].mean(axis=0) for label in np.unique(labels)])
        new_sequence_length = len(merged_sample)

        if new_sequence_length < 2:
            all_tours.append([0])  # Only one node, no tour needed
            continue

        # Step 3: Compute the sequence-based distance matrix with jumps
        dist_matrix = compute_sequence_distance_matrix_with_jumps(new_sequence_length, labels, jump_cost)

        # Step 4: Compute the MST
        mst = compute_mst(dist_matrix)

        # Step 5: Find vertices with odd degree in the MST
        odd_degree_nodes = find_odd_degree_vertices(mst)
        if len(odd_degree_nodes) % 2 != 0:
            raise ValueError("The number of vertices with odd degree is not even, which is required for perfect matching.")

        # Step 6: Compute the minimum weight perfect matching
        subgraph = nx.Graph()
        for i in range(len(odd_degree_nodes)):
            for j in range(i+1, len(odd_degree_nodes)):
                u, v = odd_degree_nodes[i], odd_degree_nodes[j]
                subgraph.add_edge(u, v, weight=dist_matrix[u, v])
        matching = compute_min_weight_matching(subgraph, dist_matrix)

        # Step 7: Combine the edges of the MST and the matching to form an Eulerian graph
        eulerian_multigraph = nx.MultiGraph(mst)
        eulerian_multigraph.add_edges_from(matching.edges(data=True))

        # Step 8: Find an Eulerian circuit
        eulerian_circuit = find_eulerian_circuit(eulerian_multigraph)

        # Step 9: Convert the Eulerian circuit to a Hamiltonian circuit by shortcutting repeated vertices
        hamiltonian_path = convert_to_hamiltonian_circuit(eulerian_circuit)

        # Step 10: Map the Hamiltonian path back to the original node indices
        original_path = [np.where(labels == label)[0][0] for label in np.unique(labels)[hamiltonian_path]]
        all_tours.append(original_path)

    return all_tours

def test_christofides_algorithm():
    samples = np.array([[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 1 - Distinct Nodes:", tours)

    samples = np.array([[[0, 0], [1, 1], [1, 1], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 2 - Duplicate Nodes:", tours)

    samples = np.array([[[0, 0]]])  # 1 sample, 1 node, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 3 - Single Node:", tours)

    samples = np.array([[[0, 0], [1, 1]]])  # 1 sample, 2 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 4 - Two Nodes:", tours)

    samples = np.array([[[1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 5 - Identical Features:", tours)

    samples = np.array([[[0, 0], [1000, 1000], [2000, 2000], [3000, 3000], [4000, 4000]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 6 - Sparse Sequence:", tours)

    samples = np.array([[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 7 - All Nodes in One Cluster:", tours)

    samples = np.array([[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=10.0, jump_cost=0.1)
    print("Test 8 - No Similar Nodes (large eps):", tours)

test_christofides_algorithm()



Processing Sample 1/1
Labels: [0 1 2 3 4]
Distance Matrix:
[[0 1 2 3 4]
 [1 0 1 2 3]
 [2 1 0 1 2]
 [3 2 1 0 1]
 [4 3 2 1 0]]
MST Edges: [(0, 1, {'weight': 1}), (1, 2, {'weight': 1}), (2, 3, {'weight': 1}), (3, 4, {'weight': 1})]
Odd Degree Nodes: [0, 4]
Matching Edges: [(4, 0, {'weight': 4})]
Eulerian Circuit: [(0, 4), (4, 3), (3, 2), (2, 1), (1, 0)]
Hamiltonian Path: [0, 4, 3, 2, 1, 0]
Test 1 - Distinct Nodes: [[0, 4, 3, 2, 1, 0]]

Processing Sample 1/1
Labels: [0 1 1 2 3]
Distance Matrix:
[[0 1 2 3]
 [1 0 0 2]
 [2 0 0 1]
 [3 2 1 0]]
MST Edges: [(0, 1, {'weight': 1}), (1, 2, {'weight': 0}), (2, 3, {'weight': 1})]
Odd Degree Nodes: [0, 3]
Matching Edges: [(3, 0, {'weight': 3})]
Eulerian Circuit: [(0, 3), (3, 2), (2, 1), (1, 0)]
Hamiltonian Path: [0, 3, 2, 1, 0]
Test 2 - Duplicate Nodes: [[0, 4, 3, 1, 0]]

Processing Sample 1/1
Labels: [0]
Test 3 - Single Node: [[0]]

Processing Sample 1/1
Labels: [0 1]
Distance Matrix:
[[0 1]
 [1 0]]
MST Edges: [(0, 1, {'weight': 1})]
Odd Degree Nodes

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import networkx as nx
from networkx.algorithms.matching import max_weight_matching
from sklearn.cluster import DBSCAN

def merge_similar_nodes(sample, eps=0.1):
    clustering = DBSCAN(eps=eps, min_samples=1).fit(sample)
    labels = clustering.labels_
    unique_labels = np.unique(labels)
    node_map = {old_idx: new_idx for new_idx, label in enumerate(unique_labels) for old_idx in np.where(labels == label)[0]}
    return node_map, labels

def compute_mst(dist_matrix):
    G = nx.Graph()
    sequence_length = dist_matrix.shape[0]
    for i in range(sequence_length):
        for j in range(i+1, sequence_length):
            G.add_edge(i, j, weight=dist_matrix[i, j])
    mst = nx.minimum_spanning_tree(G)
    return mst

def find_odd_degree_vertices(mst):
    odd_degree_nodes = [v for v in mst.nodes() if mst.degree(v) % 2 != 0]
    return odd_degree_nodes

def compute_min_weight_matching(subgraph, dist_matrix):
    min_weight_matching = max_weight_matching(subgraph, maxcardinality=True)
    matching = nx.Graph()
    for u, v in min_weight_matching:
        matching.add_edge(u, v, weight=dist_matrix[u, v])
    return matching

def find_eulerian_circuit(eulerian_multigraph):
    eulerian_circuit = list(nx.eulerian_circuit(eulerian_multigraph))
    return eulerian_circuit

def convert_to_hamiltonian_circuit(eulerian_circuit):
    hamiltonian_path = []
    visited = set()
    for u, v in eulerian_circuit:
        if u not in visited:
            hamiltonian_path.append(u)
            visited.add(u)
    if hamiltonian_path[0] != hamiltonian_path[-1]:
        hamiltonian_path.append(hamiltonian_path[0])
    return hamiltonian_path

def christofides_algorithm(samples, eps=0.1, jump_cost=0.1):
    sample_size, sequence_length, feature_size = samples.shape
    all_tours = []
    for sample_idx, sample in enumerate(samples):
        print(f"Processing Sample {sample_idx + 1}/{sample_size}")

        node_map, labels = merge_similar_nodes(sample, eps)
        print(f"Labels: {labels}")

        merged_sample = np.array([sample[labels == label].mean(axis=0) for label in np.unique(labels)])
        new_sequence_length = len(merged_sample)

        if new_sequence_length < 2:
            all_tours.append([0])  # Only one node, no tour needed
            continue

        dist_matrix = cdist(merged_sample, merged_sample, metric='euclidean') + jump_cost
        print(f"Distance Matrix:\n{dist_matrix}")

        mst = compute_mst(dist_matrix)
        print(f"MST Edges: {mst.edges(data=True)}")

        odd_degree_nodes = find_odd_degree_vertices(mst)
        print(f"Odd Degree Nodes: {odd_degree_nodes}")

        if len(odd_degree_nodes) % 2 != 0:
            raise ValueError("The number of vertices with odd degree is not even, which is required for perfect matching.")

        subgraph = nx.Graph()
        for i in range(len(odd_degree_nodes)):
            for j in range(i+1, len(odd_degree_nodes)):
                u, v = odd_degree_nodes[i], odd_degree_nodes[j]
                subgraph.add_edge(u, v, weight=dist_matrix[u, v])

        matching = compute_min_weight_matching(subgraph, dist_matrix)
        print(f"Matching Edges: {matching.edges(data=True)}")

        eulerian_multigraph = nx.MultiGraph(mst)
        eulerian_multigraph.add_edges_from(matching.edges(data=True))

        eulerian_circuit = find_eulerian_circuit(eulerian_multigraph)
        print(f"Eulerian Circuit: {eulerian_circuit}")

        hamiltonian_path = convert_to_hamiltonian_circuit(eulerian_circuit)
        print(f"Hamiltonian Path: {hamiltonian_path}")

        original_path = [np.where(labels == label)[0][0] for label in np.unique(labels)[hamiltonian_path]]
        all_tours.append(original_path)

    return all_tours

def test_christofides_algorithm():
    samples = np.array([[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 1 - Distinct Nodes:", tours)

    samples = np.array([[[0, 0], [1, 1], [1, 1], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 2 - Duplicate Nodes:", tours)

    samples = np.array([[[0, 0]]])  # 1 sample, 1 node, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 3 - Single Node:", tours)

    samples = np.array([[[0, 0], [1, 1]]])  # 1 sample, 2 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 4 - Two Nodes:", tours)

    samples = np.array([[[1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 5 - Identical Features:", tours)

    samples = np.array([[[0, 0], [1000, 1000], [2000, 2000], [3000, 3000], [4000, 4000]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 6 - Sparse Sequence:", tours)

    samples = np.array([[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=0.1, jump_cost=0.1)
    print("Test 7 - All Nodes in One Cluster:", tours)

    samples = np.array([[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4]]])  # 1 sample, 5 nodes, 2 features
    tours = christofides_algorithm(samples, eps=10.0, jump_cost=0.1)
    print("Test 8 - No Similar Nodes (large eps):", tours)

test_christofides_algorithm()


Processing Sample 1/1
Labels: [0 1 2 3 4]
Distance Matrix:
[[0.1        1.51421356 2.92842712 4.34264069 5.75685425]
 [1.51421356 0.1        1.51421356 2.92842712 4.34264069]
 [2.92842712 1.51421356 0.1        1.51421356 2.92842712]
 [4.34264069 2.92842712 1.51421356 0.1        1.51421356]
 [5.75685425 4.34264069 2.92842712 1.51421356 0.1       ]]
MST Edges: [(0, 1, {'weight': 1.5142135623730952}), (1, 2, {'weight': 1.5142135623730952}), (2, 3, {'weight': 1.5142135623730952}), (3, 4, {'weight': 1.5142135623730952})]
Odd Degree Nodes: [0, 4]
Matching Edges: [(4, 0, {'weight': 5.75685424949238})]
Eulerian Circuit: [(0, 4), (4, 3), (3, 2), (2, 1), (1, 0)]
Hamiltonian Path: [0, 4, 3, 2, 1, 0]
Test 1 - Distinct Nodes: [[0, 4, 3, 2, 1, 0]]
Processing Sample 1/1
Labels: [0 1 1 2 3]
Distance Matrix:
[[0.1        1.51421356 4.34264069 5.75685425]
 [1.51421356 0.1        2.92842712 4.34264069]
 [4.34264069 2.92842712 0.1        1.51421356]
 [5.75685425 4.34264069 1.51421356 0.1       ]]
MST Edge